In [6]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [1]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
csv_file_folder =  '../../data/input'
os.listdir(csv_file_folder)

['sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'mulliken_charges.csv',
 'train.csv',
 'test.csv',
 'structures.csv',
 'structures']

In [4]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if (f.endswith('.pkl')) and (not f.startswith('.'))]

[]

In [5]:
mytrial = []

In [ ]:
# 'scalar_coupling_constant', 'fc', 'sd','pso','dso'
for y in ['dso']:
    for t in np.arange(0,8,1):
        
        print(f'y={y}, type={t}')
        df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
        df_train['y'] = df_train[y]
        df_train = df_train[df_train['type']==t]
        n_samples = 500000 if df_train.shape[0] > 500000 else df_train.shape[0]
        df_train = df_train.sample(n_samples).reset_index(drop=True)
        df_test = pd.DataFrame()
        

        param = {
            'columns': df_train.columns.drop(['index', 'y','group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso']).tolist(),
            'cv': {
                'cls': 'KFold',
                'init':{
                    'n_splits': 5,
                    'shuffle': True,
                    'random_state': 42,
                },
            },
            'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
            'model': {
                'cls': 'lgb.LGBMRegressor',
                'init': {
                    'learning_rate': 0.2833769330240482,
                    'feature_fraction': 0.8818248470204605,
                    'bagging_fraction': 0.8205197060908092,
                    'min_data_in_leaf': 202,
                    'lambda_l1': 0.017039063121824582,
                    'lambda_l2': 0.8318702431636841,
                    'max_bin': 100,
                    'num_leaves': 255,
                    'random_state': 3895,
                    'n_jobs': 16
                },
                'fit': {}
            },
            'metric': 'mean_absolute_error'
        }



        process_queue = PQueue(df_train, df_test, param, mytrial)
        sort_features = SortFeatureSelectTopNProcess(**{'top_n':200})
        select_topn = RFESelectTopNProcess(**{'n_features_remain':20, 'n_features_to_remove':10})
        remove_useless = RFERemoveUselessFeaturesProcess(**{})
        process_queue.insert_node(sort_features)
        process_queue.insert_node(select_topn)
        process_queue.insert_node(remove_useless)

        try:
            result = process_queue.run()
        except Exception as e:
            print(e.__str__())
        print(len(process_queue.trial))
        print(process_queue.param)
    

y=dso, type=0


In [5]:
df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
df_train['y'] = df_train['pso']

In [6]:
columns_list = [{'columns': ['dist_to_type_0_mean', 'atom_1_n_bonds', 'distance_center0', 'dist_to_type_std', 'dist_to_type_1_mean', 'dist_H_3_y', 'tertiary_distance_1', 'tertiary_atom_1', 'adC1', 'distC1', 'dist_O_0_x', 'yukawa_C.x', 'dist_C_3_y', 'inv_distPR', 'eem_0', 'dist_H_1_y', 'dist_H_4_y', 'dist_C_4_y', 'tertiary_angle_0', 'distance_farthest_0', 'tertiary_angle_2', 'distance_center1', 'tertiary_angle_3', 'tertiary_atom_0', 'dist_O_1_y', 'tertiary_atom_2', 'vander_C.x', 'coulomb_H.y', 'dist_H_2_x', 'dist_C_4_x', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_H_3_x', 'dist_O_0_y', 'dist_H_2_y', 'dist_N_1_x', 'bond_atom', 'inv_dist0R', 'atom_1_bond_lengths_mean', 'inv_dist1', 'dist_C_2_y', 'adN1', 'molecule_dist_min', 'dist_H_0_y', 'atom_index_1_cycle_size_mean', 'dist_C_3_x', 'eem_1', 'dist_C_1_x', 'dist_C_0_y', 'dist_H_0_x', 'tertiary_distance_0', 'tertiary_distance_2', 'vander_H.y', 'tertiary_angle_1', 'adC2', 'dist_N_0_y', 'dist_C_0_x', 'dist_H_1_x', 'dist_H_4_x', 'molecule_name.1', 'atom_1_bond_lengths_std', 'adC3', 'tertiary_distance_5', 'yukawa_N.x', 'link0', 'dist_to_type_mean', 'molecule_atom_index_0_dist_max_div', 'dist_N_0_x', 'tertiary_atom_3', 'tertiary_distance_9', 'dist_C_1_y', 'tertiary_distance_3', 'inv_dist1R', 'molecule_atom_index_1_dist_min_diff', 'vander_O.y', 'tertiary_distance_4', 'molecule_atom_index_0_dist_mean_div', 'dist_C_2_x', 'dist_O_1_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_to_type_0_mean', 'atom_1_n_bonds', 'distance_center0', 'dist_to_type_std', 'dist_to_type_1_mean', 'dist_H_3_y', 'tertiary_distance_1', 'tertiary_atom_1', 'adC1', 'distC1', 'dist_O_0_x', 'yukawa_C.x', 'dist_C_3_y', 'inv_distPR', 'eem_0', 'dist_H_1_y', 'dist_H_4_y', 'dist_C_4_y', 'tertiary_angle_0', 'distance_farthest_0', 'tertiary_angle_2', 'distance_center1', 'tertiary_angle_3', 'tertiary_atom_0', 'dist_O_1_y', 'tertiary_atom_2', 'vander_C.x', 'coulomb_H.y', 'dist_H_2_x', 'dist_C_4_x', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_H_3_x', 'dist_O_0_y', 'dist_H_2_y', 'dist_N_1_x', 'bond_atom', 'inv_dist0R', 'atom_1_bond_lengths_mean', 'inv_dist1', 'dist_C_2_y', 'adN1', 'molecule_dist_min', 'dist_H_0_y', 'atom_index_1_cycle_size_mean', 'dist_C_3_x', 'eem_1', 'dist_C_1_x', 'dist_C_0_y', 'dist_H_0_x', 'tertiary_distance_0', 'tertiary_distance_2', 'vander_H.y', 'tertiary_angle_1', 'adC2', 'dist_N_0_y', 'dist_C_0_x', 'dist_H_1_x', 'dist_H_4_x', 'molecule_name.1', 'atom_1_bond_lengths_std', 'adC3', 'tertiary_distance_5', 'yukawa_N.x', 'link0', 'dist_to_type_mean', 'molecule_atom_index_0_dist_max_div', 'dist_N_0_x', 'tertiary_atom_3', 'tertiary_distance_9', 'dist_C_1_y', 'tertiary_distance_3', 'inv_dist1R', 'molecule_atom_index_1_dist_min_diff', 'vander_O.y', 'tertiary_distance_4', 'molecule_atom_index_0_dist_mean_div', 'dist_C_2_x', 'dist_O_1_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_to_type_0_mean', 'atom_1_n_bonds', 'distance_center0', 'dist_to_type_std', 'dist_to_type_1_mean', 'dist_H_3_y', 'tertiary_distance_1', 'tertiary_atom_1', 'adC1', 'distC1', 'dist_O_0_x', 'yukawa_C.x', 'dist_C_3_y', 'inv_distPR', 'eem_0', 'dist_H_1_y', 'dist_H_4_y', 'dist_C_4_y', 'tertiary_angle_0', 'distance_farthest_0', 'tertiary_angle_2', 'distance_center1', 'tertiary_angle_3', 'tertiary_atom_0', 'dist_O_1_y', 'tertiary_atom_2', 'vander_C.x', 'coulomb_H.y', 'dist_H_2_x', 'dist_C_4_x', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_H_3_x', 'dist_O_0_y', 'dist_H_2_y', 'dist_N_1_x', 'bond_atom', 'inv_dist0R', 'atom_1_bond_lengths_mean', 'inv_dist1', 'dist_C_2_y', 'adN1', 'molecule_dist_min', 'dist_H_0_y', 'atom_index_1_cycle_size_mean', 'dist_C_3_x', 'eem_1', 'dist_C_1_x', 'dist_C_0_y', 'dist_H_0_x', 'tertiary_distance_0', 'tertiary_distance_2', 'vander_H.y', 'tertiary_angle_1', 'adC2', 'dist_N_0_y', 'dist_C_0_x', 'dist_H_1_x', 'dist_H_4_x', 'molecule_name.1', 'atom_1_bond_lengths_std', 'adC3', 'tertiary_distance_5', 'yukawa_N.x', 'link0', 'dist_to_type_mean', 'molecule_atom_index_0_dist_max_div', 'dist_N_0_x', 'tertiary_atom_3', 'tertiary_distance_9', 'dist_C_1_y', 'tertiary_distance_3', 'inv_dist1R', 'molecule_atom_index_1_dist_min_diff', 'vander_O.y', 'tertiary_distance_4', 'molecule_atom_index_0_dist_mean_div', 'dist_C_2_x', 'dist_O_1_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_to_type_0_mean', 'tertiary_distance_16', 'tertiary_distance_15', 'dist_to_type_1_mean', 'tertiary_distance_14', 'yukawa_O.x', 'tertiary_atom_1', 'cos_c0_c1', 'dist_O_0_x', 'yukawa_C.x', 'dist_C_3_y', 'yukawa_N.y', 'dist_H_1_y', 'inv_distPR', 'tertiary_distance_11', 'coulomb_C.y', 'dist_C_4_y', 'tertiary_angle_0', 'tertiary_angle_3', 'tertiary_atom_0', 'cos_c0', 'yukawa_C.y', 'dist_xyz', 'dist_O_1_y', 'tertiary_atom_2', 'vander_C.y', 'vander_C.x', 'coulomb_H.y', 'dist_H_2_x', 'dist_C_4_x', 'tertiary_distance_10', 'dist_H_3_x', 'tertiary_distance_7', 'vander_N.y', 'tertiary_atom_19', 'dist_O_0_y', 'dist_N_1_x', 'tertiary_atom_17', 'yukawa_O.y', 'inv_dist0R', 'tertiary_distance_19', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'adN1', 'inv_distPE', 'molecule_atom_index_0_dist_min_div', 'dist_F_0_x', 'molecule_dist_min', 'yukawa_F.y', 'tertiary_distance_6', 'dist_C_3_x', 'molecule_dist_mean', 'tertiary_distance_12', 'cos_c1', 'dist_C_1_x', 'dist_C_0_y', 'tertiary_angle_1', 'adC2', 'dist_N_0_y', 'dist_C_0_x', 'dist_H_1_x', 'dist_H_4_x', 'cos_f0_f1', 'adC3', 'dist_N_1_y', 'yukawa_N.x', 'tertiary_distance_8', 'link0', 'dist_to_type_mean', 'dist_N_0_x', 'tertiary_distance_9', 'molecule_atom_index_0_dist_min_diff', 'atom_0_bond_lengths_max', 'dist_C_1_y', 'vander_O.x', 'tertiary_distance_3', 'tertiary_distance_13', 'tertiary_distance_4', 'dist_C_2_x', 'dist_O_1_x', 'atom_n'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_atom_0', 'atom_1_bond_lengths_max', 'atom_1_n_bonds', 'tertiary_atom_1', 'dist_C_0_y', 'molecule_atom_index_0_dist_max_diff', 'cos_c0', 'tertiary_angle_2', 'tertiary_distance_2', 'molecule_atom_index_0_dist_min_div', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_1', 'atom_0_bond_lengths_max', 'eem_1', 'tertiary_distance_1', 'dist_C_1_x', 'atom_index_1_cycle_size_mean', 'cos_f0', 'adC2', 'inv_dist1R', 'tertiary_atom_2', 'molecule_atom_index_1_dist_min_div', 'atom_1_bond_lengths_mean', 'inv_distPR', 'dist_to_type_1_mean', 'dist_N_0_x', 'dist_to_type_0_mean', 'dist_to_type_mean', 'dist_xyz', 'dist_H_1_y', 'dist_H_2_y', 'dist_O_0_y', 'atom_1_bond_lengths_min', 'dist_C_1_y', 'adC3', 'dist_N_0_y', 'dist_C_2_y', 'molecule_atom_index_0_dist_max_div', 'eem_0', 'dist_C_0_x', 'inv_dist0R', 'dist_C_3_y', 'cos_c0_c1', 'tertiary_distance_3', 'adN1', 'sd_molecule_atom_0_dist_xyz', 'inv_dist0E', 'dist_C_3_x', 'min_molecule_atom_1_dist_xyz', 'tertiary_angle_0', 'vander_C.y', 'distance_farthest_0', 'molecule_atom_index_0_dist_mean_div', 'molecule_type_dist_max', 'atom_1_bond_lengths_std', 'tertiary_angle_3', 'min_molecule_atom_0_dist_xyz', 'dist_C_2_x', 'dist_H_0_x', 'dist_O_0_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_atom_0', 'atom_1_bond_lengths_max', 'atom_1_n_bonds', 'tertiary_atom_1', 'dist_C_0_y', 'molecule_atom_index_0_dist_max_diff', 'cos_c0', 'tertiary_angle_2', 'tertiary_distance_2', 'molecule_atom_index_0_dist_min_div', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_1', 'atom_0_bond_lengths_max', 'eem_1', 'tertiary_distance_1', 'dist_C_1_x', 'atom_index_1_cycle_size_mean', 'cos_f0', 'adC2', 'inv_dist1R', 'tertiary_atom_2', 'molecule_atom_index_1_dist_min_div', 'atom_1_bond_lengths_mean', 'inv_distPR', 'dist_to_type_1_mean', 'dist_N_0_x', 'dist_to_type_0_mean', 'dist_to_type_mean', 'dist_xyz', 'dist_H_1_y', 'dist_H_2_y', 'dist_O_0_y', 'atom_1_bond_lengths_min', 'dist_C_1_y', 'adC3', 'dist_N_0_y', 'dist_C_2_y', 'molecule_atom_index_0_dist_max_div', 'eem_0', 'dist_C_0_x', 'inv_dist0R', 'dist_C_3_y', 'cos_c0_c1', 'tertiary_distance_3', 'adN1', 'sd_molecule_atom_0_dist_xyz', 'inv_dist0E', 'dist_C_3_x', 'min_molecule_atom_1_dist_xyz', 'tertiary_angle_0', 'vander_C.y', 'distance_farthest_0', 'molecule_atom_index_0_dist_mean_div', 'molecule_type_dist_max', 'atom_1_bond_lengths_std', 'tertiary_angle_3', 'min_molecule_atom_0_dist_xyz', 'dist_C_2_x', 'dist_H_0_x', 'dist_O_0_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['adC1', 'yukawa_C.x', 'dist_C_2_y', 'yukawa_F.y', 'molecule_atom_1_dist_min_div', 'molecule_couples', 'dist_to_type_mean', 'molecule_atom_index_0_dist_min_diff', 'dist_C_3_y', 'tertiary_distance_12', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_11', 'min_molecule_atom_0_dist_xyz', 'dist_C_0_x', 'cos_c0', 'dist_O_1_x', 'yukawa_O.x', 'dist_N_1_x', 'yukawa_N.y', 'tertiary_distance_16', 'dist_C_0_y', 'vander_C.y', 'dist_to_type_std', 'tertiary_atom_16', 'yukawa_O.y', 'dist_O_0_x', 'yukawa_C.y', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_distance_13', 'link0', 'dist_N_0_y', 'dist_O_0_y', 'inv_dist0R', 'tertiary_angle_2', 'vander_C.x', 'tertiary_angle_1', 'dist_C_4_x', 'dist_O_1_y', 'dist_xyz', 'tertiary_distance_6', 'dist_N_1_y', 'dist_N_0_x', 'molecule_name.1', 'dist_C_2_x', 'molecule_atom_index_1_dist_std_diff', 'distance_f1', 'cos_c0_c1', 'cos_c1', 'tertiary_distance_10', 'tertiary_distance_9', 'tertiary_atom_15', 'dist_C_1_x', 'molecule_atom_1_dist_min_diff', 'yukawa_H.y', 'tertiary_atom_17', 'tertiary_angle_0', 'inv_distPR', 'distance_c1', 'coulomb_H.y', 'dist_to_type_1_mean', 'atom_1_bond_lengths_mean', 'tertiary_distance_7', 'dist_H_0_x', 'dist_H_4_x', 'tertiary_distance_3', 'tertiary_distance_14', 'max_molecule_atom_0_dist_xyz', 'dist_to_type_0_mean', 'dist_O_2_y', 'tertiary_distance_8', 'dist_C_1_y', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_1', 'tertiary_angle_3', 'dist_C_3_x', 'molecule_dist_mean', 'dist_C_4_y', 'dist_H_0_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['adC1', 'dist_C_2_y', 'dist_H_1_y', 'tertiary_distance_5', 'tertiary_angle_4', 'tertiary_distance_2', 'dist_to_type_mean', 'tertiary_distance_4', 'molecule_atom_index_0_dist_min_diff', 'dist_C_3_y', 'min_molecule_atom_0_dist_xyz', 'dist_C_0_x', 'vander_N.y', 'cos_c0', 'dist_H_1_x', 'eem_1', 'dist_N_1_x', 'atom_0_bond_lengths_max', 'dist_C_0_y', 'dist_to_type_std', 'tertiary_atom_3', 'atom_1_bond_lengths_max', 'inv_distPE', 'distance_farthest_0', 'dist_O_0_x', 'molecule_atom_index_0_dist_max_div', 'adC3', 'yukawa_C.y', 'adC2', 'molecule_atom_index_0_dist_mean_diff', 'dist_N_0_y', 'dist_O_0_y', 'vander_O.y', 'cos_f0', 'inv_dist0R', 'inv_dist1R', 'tertiary_angle_2', 'atom_1_bond_lengths_std', 'yukawa_N.x', 'tertiary_angle_1', 'dist_C_4_x', 'dist_xyz', 'tertiary_distance_6', 'dist_H_2_y', 'dist_N_1_y', 'dist_N_0_x', 'molecule_atom_index_0_dist_max_diff', 'dist_C_2_x', 'molecule_atom_index_1_dist_std_diff', 'cos_c0_c1', 'cos_c1', 'inv_dist1', 'tertiary_distance_10', 'dist_C_1_x', 'tertiary_angle_0', 'inv_distPR', 'dist_to_type_1_mean', 'atom_1_bond_lengths_mean', 'atom_1_n_bonds', 'min_molecule_atom_1_dist_xyz', 'dist_H_0_x', 'tertiary_distance_3', 'tertiary_atom_1', 'atom_index_1_cycle_size_mean', 'dist_to_type_0_mean', 'coulomb_H.x', 'mean_molecule_atom_0_dist_xyz', 'dist_C_1_y', 'distN0', 'molecule_atom_index_1_dist_min_diff', 'molecule_atom_index_0_dist_min_div', 'vander_N.x', 'tertiary_angle_3', 'tertiary_distance_1', 'dist_C_3_x', 'tertiary_atom_0', 'dist_H_0_y', 'molecule_type_0_dist_std_diff'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}]

In [7]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), columns_list[t], f'modeling for {t}', df_test=df_test[df_test['type']==t].reset_index(drop=True), trial=mytrial, is_output_feature_importance=False, trial_level=1)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [8]:
df_trial = pd.DataFrame(mytrial)
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'message']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,message
0,2019-07-21 06:57:41.650499,modeling for 0,79,0.017113,0.019435,0.002322,modeling for 0
1,2019-07-21 06:58:09.703872,modeling for 3,81,0.011039,0.013633,0.002594,modeling for 3
2,2019-07-21 06:58:12.924737,modeling for 1,79,0.016607,0.026035,0.009428,modeling for 1
3,2019-07-21 06:58:20.423905,modeling for 4,60,0.007376,0.010555,0.003179,modeling for 4
4,2019-07-21 06:59:35.950985,modeling for 2,79,0.018547,0.020192,0.001644,modeling for 2
5,2019-07-21 07:00:20.553231,modeling for 6,78,0.015375,0.017710,0.002335,modeling for 6
6,2019-07-21 07:01:41.406375,modeling for 5,60,0.013618,0.014411,0.000793,modeling for 5
7,2019-07-21 07:01:52.741052,modeling for 7,78,0.005663,0.007964,0.002301,modeling for 7


In [9]:
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
for t in df_train.type.unique().tolist():
    index = df_train[df_train['type']==t]['index'].values
    print(t, mean_absolute_error(df_valid_pred[df_valid_pred['index'].isin(index)].sort_values(by=['index']).reset_index(drop=True).predict.values, df_train[df_train['index'].isin(index)].reset_index(drop=True).y.values))

0.016725810061076055
0 0.019434655292000644
3 0.013632507174711326
1 0.02603495341178363
4 0.010555004939426493
2 0.020191539020838728
6 0.01771015673229714
5 0.014410694456529377
7 0.007964372967414476


In [10]:
df_test_pred.tail()

,index,0,1,2,3,4
2505537,7163684,0.201777,0.197098,0.148143,0.183998,0.168900
2505538,7163685,0.108909,0.142198,0.119749,0.113101,0.131184
2505539,7163686,0.314042,0.325877,0.310543,0.327193,0.337018
2505540,7163687,0.029297,0.057392,0.043731,0.016709,0.028348
2505541,7163688,0.454893,0.407355,0.371889,0.468198,0.430555


In [11]:
# df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['pso'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['id'] = df_test_pred['index']
df_submit.to_pickle('../../data/oof/lgbm_pso_test.pkl')

In [12]:
df_submit = df_valid_pred.rename(columns={'predict':'pso', 'index':'id'})
df_submit.to_pickle('../../data/oof/lgbm_pso_train.pkl')